# Start using the spotify API

In [ ]:
import config
import spotipy
import json
import requests
from spotipy.oauth2 import SpotifyClientCredentials
import pprint
import pandas as pd
import base64

In [ ]:
# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(
    client_id=config.client_id,
    client_secret=config.client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
results = sp.search(q="Lose yourself",limit=3,market="GB")

In [ ]:
pprint.pprint(track_features)

In [ ]:
track_id=results["tracks"]["items"][0]["id"]

# Get "Today's Top Hits" playlist ID
playlist_id = "37i9dQZF1DXcBWIGoYBM5M"

# Get top tracks and show only the ids
playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=10)

track_id = []
track_features = []
# Print top tracks
for i, item in enumerate(playlist_items["items"]):
    # get track_id
    track_id = item["track"]["id"]
    # get features and store them in a list
    track_features.append(sp.audio_features(track_id)[0])

# create a dataframe from the features listS
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

In [ ]:
# create a list containing all marketsS
markets = list(sp.available_markets()["markets"])

In [ ]:
# get listS of category IDs for all the markets
category_id_lst = []
for market in markets:
    # get list of category IDs for a certain country
    categories_per_country = sp.categories(country=market, limit=50)["categories"]["items"]
    # get to the ID's of the categories and store them in a list
    for category in categories_per_country:
        if not category["id"] in category_id_lst:
            category_id_lst.append(category["id"])
        else:
            print("already in list")

In [ ]:
# create a list of playlist IDs from all the categories of all the markets
playlist_id_lst = []
for category_id in category_id_lst:
    try:
        playlists_per_genre = sp.category_playlists(category_id=category_id, limit=50)["playlists"]["items"]
        try:
            for playlist in playlists_per_genre:
                if not playlist["id"] in playlist_id_lst:
                    playlist_id_lst.append(playlist["id"])
                else:
                    print("already in list")
        except:
            nothing = True
    except spotipy.SpotifyException as e:
        print("An error occurred: ")

In [239]:
len(playlist_id_lst)

1360

In [240]:
user = "spotify"
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [245]:
# Get top tracks from all the playlists
for playlist_id in playlist_id_lst:
    playlist_items = sp.playlist_items(playlist_id, fields="items.track.id", limit=1)

    # now get the ID's
    track_id = []
    track_features = []
    # Print top tracks
    for i, item in enumerate(playlist_items["items"]):
        try:
            # get track_id
            track_id = item["track"]["id"]
            # get features and store them in a list
            track_features.append(sp.audio_features(track_id)[0])
        except TypeError:
            print("type error")

# create a dataframe from the features listS
df = pd.DataFrame(track_features)
df = df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]

type error
type error


ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [246]:
df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.707,0.681,-4.325,0.0668,0.0632,0.000005,0.0322,0.646,117.999,0yLdNVWF3Srea0uzk55zFn,200455
1,0.644,0.735,-5.747,0.0391,0.0521,0.144000,0.1610,0.418,88.980,1Qrg8KqiBpW07V7PNxwwwL,153947
2,0.696,0.809,-8.254,0.0500,0.2520,0.000128,0.2480,0.857,132.962,6AQbmUe0Qwf5PZnt4HmTXv,131013
3,0.538,0.742,-5.355,0.1140,0.1380,0.000047,0.0934,0.250,96.107,5Z2MiIZ5I3jJvvmeWMLbOQ,272373
4,0.662,0.398,-6.691,0.0275,0.7160,0.000000,0.1110,0.468,107.001,4sx6NRwL6Ol3V6m9exwGlQ,145850
5,0.715,0.620,-6.005,0.0484,0.4170,0.000000,0.0822,0.172,97.950,2dHHgzDwk4BJdRwy9uXhTO,221520
6,0.778,0.632,-5.600,0.0493,0.2740,0.000000,0.0915,0.498,122.104,4nrPB8O7Y7wsOCJdgXkthe,218289
7,0.801,0.806,-5.206,0.0381,0.3820,0.000669,0.1140,0.802,106.999,0WtM2NBVQNNJLh6scP13H8,239318
8,0.520,0.731,-5.338,0.0557,0.3420,0.001010,0.3110,0.662,173.930,4LRPiXqCikLlN15c3yImP7,167303
9,0.714,0.472,-7.375,0.0864,0.0130,0.000005,0.2660,0.238,131.121,3nqQXoyQOWXiESFLlDF1hG,156943


# code snippets

In [242]:
# Set up authorization headers
client_id = config.client_id
client_secret = config.client_secret
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode("ascii")).decode("ascii")
headers = {"Authorization": f"Basic {auth_header}"}

# Get access token
url = "https://accounts.spotify.com/api/token"
data = {"grant_type": "client_credentials"}
response = requests.post(url, headers=headers, data=data)
access_token = response.json()["access_token"]

# Make request to Top Tracks endpoint
artist_id = "0OdUWJ0sBjDrqHygGUXeCF"  # example artist ID
url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
headers = {"Authorization": f"Bearer {access_token}"}
params = {"market": "US"}

response = requests.get(url, headers=headers, params=params)